# Support Vector Machines : Image Classification

# Data Set: The CIFAR-10 dataset
The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another. Between them, the training batches contain exactly 5000 images from each class.


Link: https://www.cs.toronto.edu/~kriz/cifar.html

In [23]:
import tarfile
import os
import pickle
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
from PIL import Image


In [24]:
# Path to the tar.gz file
file_path = '/home/mthobisi/Downloads/cifar-10-python.tar.gz'
extracted_dir_path = '/home/mthobisi/Downloads/cifar-10-python'

# Extract the tar.gz file
if not os.path.exists(extracted_dir_path):
    with tarfile.open(file_path, 'r:gz') as tar:
        tar.extractall(path=extracted_dir_path)

In [25]:
def load_batch(batch_path):
    with open(batch_path, 'rb') as file:
        batch = pickle.load(file, encoding='latin1')
    data = batch['data']
    labels = batch['labels']
    return data, labels

In [35]:
def load_label_names():
    with open(os.path.join(extracted_dir_path, 'cifar-10-batches-py', 'batches.meta'), 'rb') as file:
        label_names = pickle.load(file, encoding='latin1')['label_names']
    return label_names

label_names = load_label_names()
print("Label names:", label_names)

Label names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [26]:
# Load all training batches
train_data = []
train_labels = []
for i in range(1, 6):
    data, labels = load_batch(os.path.join(extracted_dir_path, 'cifar-10-batches-py', f'data_batch_{i}'))
    train_data.append(data)
    train_labels.append(labels)

In [27]:
# Combine the training batches
X_train = np.concatenate(train_data)
y_train = np.concatenate(train_labels)

In [28]:
# Load the test batch
X_test, y_test = load_batch(os.path.join(extracted_dir_path, 'cifar-10-batches-py', 'test_batch'))

In [29]:
# Reshape the labels arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

In [30]:
# Filter the training data
dog_cat_train_mask = np.isin(y_train, [3, 5])
X_train_dog_cat = X_train[dog_cat_train_mask]
y_train_dog_cat = y_train[dog_cat_train_mask]

# Filter the test data
dog_cat_test_mask = np.isin(y_test, [3, 5])
X_test_dog_cat = X_test[dog_cat_test_mask]
y_test_dog_cat = y_test[dog_cat_test_mask]

# Convert class labels to binary: 0 for cat, 1 for dog
y_train_dog_cat = np.where(y_train_dog_cat == 3, 0, 1)
y_test_dog_cat = np.where(y_test_dog_cat == 3, 0, 1)


In [31]:
def preprocess_images_pil(images):
    images = images.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (num_samples, height, width, channels)
    processed_images = []
    for img in images:
        img = Image.fromarray(img)
        gray = img.convert('L')  # Convert to grayscale
        processed_images.append(np.array(gray).flatten())  # Flatten the image
    return np.array(processed_images)

# Preprocess the training and testing images
X_train_processed = preprocess_images_pil(X_train_dog_cat)
X_test_processed = preprocess_images_pil(X_test_dog_cat)


In [32]:
# Standardize the data
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train_processed)
X_test_processed = scaler.transform(X_test_processed)


In [33]:
# Train the SVM model
model = SVC(kernel='rbf', C=1.0, gamma='scale')
model.fit(X_train_processed, y_train_dog_cat)

SVC()

In [34]:
# Make predictions
y_pred = model.predict(X_test_processed)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test_dog_cat, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_dog_cat, y_pred))
print("Classification Report:\n", classification_report(y_test_dog_cat, y_pred))


Accuracy: 0.654
Confusion Matrix:
 [[670 330]
 [362 638]]
Classification Report:
               precision    recall  f1-score   support

           0       0.65      0.67      0.66      1000
           1       0.66      0.64      0.65      1000

    accuracy                           0.65      2000
   macro avg       0.65      0.65      0.65      2000
weighted avg       0.65      0.65      0.65      2000



In [36]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
scores = cross_val_score(SVC(kernel='rbf', C=1.0, gamma='scale'), X_train_processed, y_train_dog_cat, cv=5)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", np.mean(scores))
